In [1]:
import numpy as np
import pandas as pd

from scipy import stats
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
from IPython.display import display

In [2]:
def get_n(p_c:float, p_t:float, sigma_c:float, sigma_t:float,
          alpha:float, beta:float, orientation:str, plot:bool=True) -> (int, int): 
    """
    Функция нахождения требуемого количества наблюдений исходя
    из извесных средних значений распределений, их дисперсий, alpha, beta.
    
    Возвращает размеры контрольной и тритмент выборок, нужные для проведения теста
    -------------------------------------------------------
    p_c - среднее контрольного распределения
    p_t - среднее тритмент распределения
    sigma_c - дисперсия контрольного распределения
    sigma_t - дисперсия тритмент распределения
    alpha - уровень ошибки 1го рода
    beta - уровень ошибки 2го рода
    orientation - в каком направлении будет проверяться гипотеза ('left', 'right', 'two_sided')
    plot - флаг построения графиков при выполнении функции
    """ 
    # проверка того, что в функцию передано корректное значение направления теста
    if orientation not in ('left', 'right', 'two_sided'):
        raise ValueError('Unknown orientation type')
        
    # обработка случая, когда средние распределений равны
    if p_c == p_t:
        print('Equal mean values of given distributions. Infinity amount of observations needed')
        return 10000, 10000 # хардкодинг количестванаблюдений при равенстве средних распределений (для тестирования параметров АБ)
    
    # расчет абсолютного значения mde
    mde = p_t - p_c
    
    # оценка q - параметра для оптимальной разбивки по группам (для расчета количества наблюдений в каждой группе)
    # формула выведена через минимизацию дисперсии случайной величины p1 - p2
    q = sigma_c / (sigma_c + sigma_t)
    
    if orientation == 'left':
        alpha_procentile = alpha
        beta_procentile = beta
    elif orientation == 'right':
        alpha_procentile = (1 - alpha)
        beta_procentile = (1 - beta)
    else: # orientaion == 'two_sided'
        if p_c > p_t:
            alpha_procentile = alpha / 2
            beta_procentile = beta
        else: # p_c < p_t
            alpha_procentile = (1 - alpha / 2)
            beta_procentile = (1 - beta)
    
    z_alpha = stats.norm.ppf(alpha_procentile) # кванитль для Z_1-alpha
    z_beta = stats.norm.ppf(beta_procentile) # кванитль для Z_1-beta           
    
    # оценка количества наблюдений для проведения эксперимента
    n = round(((z_alpha + z_beta) / mde) ** 2 * (sigma_c ** 2 / q + sigma_t ** 2 / (1-q)))
    
    # требуемое количество наблюдений в control группе
    n_c = round(q * n)
              
    # требуемое количество наблюдений в treatment группе
    n_t = round((1 - q) * n) 
    
    if plot:
        norm_c = stats.norm(p_c, sigma_c / np.sqrt(n_c))
        norm_t = stats.norm(p_t, sigma_t / np.sqrt(n_t))
        
        if orientation == 'left':
            z_alpha_c = norm_c.ppf(alpha)
            z_beta_t = norm_t.ppf(1 - beta)
        elif orientation == 'right':
            z_alpha_c = norm_c.ppf(1 - alpha)
            z_beta_t = norm_t.ppf(beta)
        else: # orientation == 'two_sided'
            if p_c > p_t:
                z_alpha_c = norm_c.ppf(alpha / 2)
                z_beta_t = norm_t.ppf(1 - beta)
            else: # p_c < p_t
                z_alpha_c = norm_c.ppf(1 - alpha / 2)
                z_beta_t = norm_t.ppf(beta)
            

        x_c = np.linspace(norm_c.ppf(0.0001), norm_c.ppf(0.9999), 100)
        y_c = norm_c.pdf(x_c)

        x_t = np.linspace(norm_t.ppf(0.0001), norm_t.ppf(0.9999), 100)
        y_t = norm_t.pdf(x_t)

        fig, ax = plt.subplots(1, 1)
        fig.set_size_inches(10, 5)
        ax.plot(x_c, y_c, label='norm_distirbution_c')
        ax.plot(x_t, y_t, label='norm_distirbution_t')
        plt.axvline(z_alpha_c, color='red', linestyle='--', label='z_alpha_c')
        plt.axvline(z_beta_t, color='green', linestyle='--', label='z_beta_t')
        plt.title('Probability density function of normalized sample mean')
        plt.xlabel('$x$')
        plt.ylabel('$f(x)$')
        plt.legend()
        plt.show()        
              
    return n_c, n_t

In [3]:
def z_test(c_mean_hat:float, t_mean_hat:float, var_m_c:float, var_m_t:float,
                alpha:float, orientation:str, plot:bool=True) -> (bool, float):
    """
    Функция для проверки нулевой гипотезы о разнице средних через ассимптотический АБ тест
    
    Возвращает булево значение о верности нулевой гипотезы и p_value
    -------------------------------------------------------
    c_mean_hat - выборочное среднее контрольной группы
    t_mean_hat - выборочное среднее тритмент группы
    var_m_c - дисперсия среднего контрольной группы
    var_m_t - дисперсия среднего тритмент группы
    alpha - уровень ошибки 1го рода
    orientation - в каком направлении будет проверяться гипотеза ('left', 'right', 'two_sided')
    plot - флаг построения графиков при выполнении функции
    """
    
    # путь через z-критерий
    # нормирование случайной величины разницы средних
    z = (t_mean_hat - c_mean_hat) / np.sqrt(var_m_t + var_m_c)
    
    if orientation == 'right': 
        p_value = stats.norm.sf(z)
    elif orientation == 'left':
        p_value = stats.norm.cdf(z)
    elif orientation == 'two_sided':
        alpha /= 2
        if z >= 0:
            p_value = stats.norm.sf(z)
        else:
            p_value = stats.norm.cdf(z)
    else:
        raise ValueError('Unknown orientation')
        
    accept = False # флаг, принимаем ли мы нулевую гипотезу в итоге   
    if p_value >= alpha:
        accept = True
            
    if plot:
        if accept:
            print(f'Принимаем нулевую гипотезу о равенстве средних. p_value = {p_value}')
        else:
            print(f'Отвергаем нулевую гипотезу о равенстве средних. p_value = {p_value}')
            
        x = np.linspace(stats.norm.ppf(0.001), stats.norm.ppf(0.999), 100)
        pdf_curve = stats.norm.pdf(x)

        fig, ax = plt.subplots(1, 1)
        fig.set_size_inches(10, 5)
        ax.plot(x, pdf_curve, label='norm_distirbution')
        plt.axvline(z, color='blue', linestyle='-.', label = 'z')
        
        if orientation in ['left', 'right']:
            if orientation == 'left':
                z_alpha = stats.norm().ppf(alpha)
                label = f'{alpha} quantile'
            else:
                z_alpha = stats.norm().ppf(1 - alpha)
                label = f'{1 - alpha} quantile'
            plt.axvline(z_alpha, color='red', linestyle='dashed', label=label)
            
        else:
            left, right = stats.norm.interval(1 - alpha)
            plt.axvline(left, color='red', linestyle='-.', label = f'{alpha / 2} quantile')
            plt.axvline(right, color='red', linestyle='-.', label = f'{1 - alpha / 2} quantile')
            
        if orientation == 'left' or (orientation == 'two_sided' and z <= 0):
            if stats.norm.ppf(0.001) <= z:
                xq = np.linspace(stats.norm.ppf(0.001), z, 100)
            else:
                xq = np.linspace(z, z, 1)
        else:
            if stats.norm.ppf(0.999) >= z:
                xq = np.linspace(z, stats.norm.ppf(0.999), 100)
            else:
                xq = np.linspace(z, z, 1)              

        yq = stats.norm.pdf(xq) 
        plt.fill_between(xq, 0, yq, color='blue', alpha=0.3)
        
        y_max = plt.ylim()[1]
        text_margin = 0.05
        
        plt.text(z + text_margin, 0.8*y_max, f'p_vlaue: {round(p_value, 2)}', color="blue", fontsize=10)
        plt.title(f'bern_z_test | Probability density function (Z-test, alpha: {alpha})')
        plt.xlabel('$x$')
        plt.ylabel('$pdf(x)$')
        plt.legend(loc='upper left')
        plt.show()
        
    return accept, p_value

In [4]:
def observations_count(p_c, p_t, sigma_c, sigma_t, orientation):
    """
    Функция для подсчета размера выборки для проведения A/B теста для распределения Бернулли
    в зависимости от текущего параметра среднего распределения и желаемого MDE
    
    Возвращает таблицу с нужным количеством наблюдений для различный сочетаний alpha и beta
    """
    alphas = [0.001, 0.01, 0.02, 0.05, 0.1, 0.2]
    betas = [0.001, 0.01, 0.02, 0.05, 0.1, 0.2]

    observations = [ ]
    for alpha in alphas:
        observations_cur = [ ]
        for beta in betas:
            o = sum(get_n(p_c, p_t, sigma_c, sigma_t, alpha, beta, orientation, plot=False))
            observations_cur.append(o)
        observations.append(observations_cur)

    df_e = pd.DataFrame(observations)
    df_e.columns = alphas
    df_e.index= betas
    return df_e